In [2]:
import pandas as pd
file_path = 'Meteorite_Landings.csv' 
meteorite_data = pd.read_csv(file_path)

In [3]:
from Modules.meteorite_classification import classify_meteorites
meteorite_data['classification_group'] = meteorite_data['recclass'].apply(classify_meteorites)

# **Input map with a Search function**

After extensive computation, we have reached a limitation: the display of colors and clusters does not work well with the search option, or it requires advanced coding skills. Therefore,  an input function has been included to obtain sample of the data for it to run smoothly.

Additionally, a new feature has been added: you no longer need to click on points to see their names; simply hovering over them will display the names.  

A second code and map are available, which analyze the sample to count the meteorites that have been randomly selected.


In [11]:
from Modules.inputMap import *
m = create_map_with_input(meteorite_data)

m

Enter the number of random samples you want to display:  200


In [12]:
import folium
from folium.plugins import Search
import pandas as pd

def create_map_with_input(meteorite_data):
    n_samples = int(input("Enter the number of random samples you want to display: "))  # User enters the number of points

    df_filtered = meteorite_data.dropna(subset=['reclat', 'reclong'])
    df_filtered = df_filtered[(df_filtered['reclat'] != 0) | (df_filtered['reclong'] != 0)]
    df_filtered['year'] = df_filtered['year'].fillna(0).astype(int)
    df_filtered = df_filtered[(df_filtered['reclat'] >= -90) & (df_filtered['reclat'] <= 90) &
                              (df_filtered['reclong'] >= -180) & (df_filtered['reclong'] <= 180)]

    df_sampled = df_filtered.sample(n=min(n_samples, len(df_filtered)), random_state=None)

    # Count the number of each classification in the sample
    classification_counts = df_sampled['classification_group'].value_counts()

    print("Classification counts in the sampled data:")
    print(classification_counts)

    m = folium.Map(location=[0, 0], zoom_start=2)

    classification_colors = {
        'OC': 'blue', 'CC': 'green', 'EC': 'red', 'IR': 'purple',
        'AC': 'orange', 'SI': 'darkred', 'PA': 'lightblue',
        'KC': 'darkgreen', 'Unclassified/ ungrouped meteorites': 'pink',
        'Unknown': 'gray', 'Relict': 'cadetblue', 'Lunar meteorite': 'black'
    }

    feature_group = folium.FeatureGroup(name="Meteorites").add_to(m)
    for idx, row in df_sampled.iterrows():
        popup_text = f"{row['name']}<br>Type: {row['classification_group']}<br>Mass: {row['mass (g)']}g<br>Year: {row['year']}"
        marker_color = classification_colors.get(row['classification_group'], 'gray')
        feature = folium.GeoJson(
            data={
                "type": "Feature",
                "properties": {
                    "name": row['name'],
                    "popup": popup_text,
                    "style": {"color": marker_color, "fillColor": marker_color}
                },
                "geometry": {
                    "type": "Point",
                    "coordinates": [row['reclong'], row['reclat']]
                }
            },
            tooltip=row['name'],
            style_function=lambda x: {"color": x["properties"]["style"]["color"]}
        ).add_child(folium.Popup(popup_text))
        
        feature_group.add_child(feature)

    search = Search(
        layer=feature_group,
        search_label='name',
        placeholder='Search for Meteorite Name',
        collapsed=True,
        position='topright'
    ).add_to(m)

    return m
m = create_map_with_input(meteorite_data)

m

Enter the number of random samples you want to display:  200


Classification counts in the sampled data:
classification_group
OC                                    181
IR                                      8
EC                                      5
CC                                      4
Unclassified/ ungrouped meteorites      1
AC                                      1
Name: count, dtype: int64
